In [ ]:
from google.adk.agents import LlmAgent, Agent, SequentialAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner, InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory, preload_memory, google_search, AgentTool
from google.genai import types
from dotenv import load_dotenv
import os

print("✅ ADK components imported successfully.")

In [ ]:
load_dotenv()

In [ ]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

## Agent to improvise the user prompt

In [ ]:
prompt_improve_agent = Agent(
    name="prompt_improve_agent",
    model=Gemini(model="gemini-2.5-pro", retry_options=retry_config),
    description="You are an expert prompt optimizer that transforms a simple job search prompt into a detailed, effective prompt.",
    instruction="Your task is to just improvise the given prompt with more detailed and effective words. You are not allowed to search for job openings. While generating the prompt do not any informative text. Stick only to the prompt as I need to pass it to the job_searching_agent. Add a LIMIT of MAX 5 job urls.",
    output_key="improved_prompt",
    # tools=[google_search],
)